In [48]:
!nvidia-smi

In [ ]:
# Install the Ultralytics package
!pip install ultralytics

# Check installation (optional)
import ultralytics
ultralytics.checks()

In [21]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
!ls /content/drive/MyDrive/yolo

In [4]:
import yaml

# --- MODIFY THESE PATHS ---
# Path to the root directory of your dataset (containing 'images' and 'labels' folders)
# Adjust this path based on how you loaded your data (e.g., '/content/dataset' or '/content/drive/MyDrive/my_yolo_dataset')
dataset_root_dir = '/content/drive/MyDrive/yolo' # CHANGE THIS

# Define the dataset configuration
data_yaml = {
    'path': dataset_root_dir,  # dataset root directory
    'train': 'train',  # train images (relative to 'path')
    'val': 'valid',      # val images (relative to 'path')
    # 'test': 'images/test', # test images (optional)

    # --- MODIFY CLASS INFORMATION ---
    'nc': 9,  # number of classes
    'names': ['button', 'card', 'field', 'heading', 'icon', 'image', 'link', 'paragraph', 'text']
    # Example: 'names': ['person', 'car', 'traffic_light']
}

# --- SPECIFY WHERE TO SAVE THE YAML FILE ---
# Make sure the directory exists if it's not the root
yaml_file_path = 'data.yaml' # You can place this anywhere, e.g., inside your dataset folder

# Write the dictionary to the YAML file
with open(yaml_file_path, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=False)

print(f"Created {yaml_file_path}")
# You can print the content to verify
!cat {yaml_file_path}

In [9]:
!yolo train model={model_variant} \
            data={data_config_path} \
            epochs=300 \
            imgsz=640 \
            batch=16 \
            name='my_yolo_training_run' \
            patience=10

# --- Explanation of Changes ---
# epochs=300:   Increased the maximum number of epochs to 300. Training might finish earlier due to 'patience'.
# imgsz=640:    Kept the image size the same.
# batch=16:     Kept the batch size the same. Adjust if you have memory issues or capacity.
# name='my_yolo_training_run': Changed the run name slightly to reflect the new settings and avoid overwriting previous runs.
# patience=50:  Added the early stopping patience. Training will stop automatically if the primary validation metric (usually mAP50-95) does not improve for 50 consecutive epochs. The model saved as 'best.pt' will be from the epoch with the highest validation score.

In [10]:
ls /content/drive/MyDrive/yolo/

In [12]:
# --- Define the path to your BEST trained weights ---
# Make sure this matches the path from your validation output
model_path = 'runs/detect/my_yolo_training_run/weights/best.pt'

# --- Define the path/URL/ID of your input source ---
# >>> CHANGE THIS to your actual image, video, folder, URL, or 0 for webcam <<<
source_path = '/content/drive/MyDrive/yolo/predict.png'
# Example alternatives:
# source_path = 'path/to/your/test_video.mp4'
# source_path = 'path/to/your/test_images_folder/'
# source_path = 'https://ultralytics.com/images/bus.jpg'
# source_path = 0 # For webcam

# --- Run Prediction ---
!yolo predict model={model_path} source='{source_path}' imgsz=640 conf=0.25 save=True

# --- Explanation of Arguments ---
# model: Path to your trained .pt file.
# source: The input image, video, directory, URL, or webcam ID. Use quotes around the path/URL.
# imgsz: Image size for inference. Usually best to keep it the same as training (e.g., 640).
# conf: Confidence threshold. Only detections with a score higher than this value will be kept (e.g., 0.25 means 25% confidence). Adjust as needed.
# save: If True, saves the output image/video with bounding boxes drawn on it.
# save_txt: (Optional) If True, saves results as .txt files (one per image) in YOLO format.
# save_conf: (Optional) If True and save_txt=True, includes confidence scores in the .txt files.
# show: (Optional) If True, tries to display the output image/video stream directly (might not always work smoothly in Colab).

In [14]:
from PIL import Image # For opening the image